## Solve PiCross with regex
Use numbers to represent cell state. 0 for unknown, 1 for false, 2 for true. Turn grid into string and use regex to match pattern.

In [56]:
import re
from itertools import repeat
from copy import deepcopy
from pprint import pprint

In [139]:
def init_grid(x, y):
    '''
    Initialize the grid: full 0
    '''
    grid = []
    for i in range(y):
        grid.append(list(repeat(0,x))) # why not repeat: repear returns THE SAME OBJECT
    return grid

In [203]:
x = 10
y = 10
grid = init_grid(x, y)

In [142]:
grid

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [43]:
param_hor = [[2,1], [3,1], [3,1], [1,1,2], [5,2], [2,3,2], [7,2], [6,3], [9], [3,6]]
param_ver = [[2,1], [6], [7], [1,3], [3,6], [10], [2,2,2], [3], [10],[7]]

In [6]:
assert len(param_hor) == x
assert len(param_ver) == y

### First round
Fill full lines.

In [44]:
# top down
def first_round_topdown(grid, x, param_ver):
    _grid = deepcopy(grid)
    _param_ver = deepcopy(param_ver)
    
    for i, param in enumerate(_param_ver):
         if sum(param) + len(param) - 1 == x: 
            ele = param.pop(0)
            new_grid = list(repeat(2, ele))
            while param:
                new_grid.append(1)
                ele = param.pop(0)
                new_grid.extend(list(repeat(2, ele)))
            _grid[i] = new_grid
            
    return _grid

In [147]:
# left to right
def first_round_horizontal(grid, y, param_hor):
    _grid = deepcopy(grid)
    _param_hor = deepcopy(param_hor)
    
    for i, param in enumerate(_param_hor):
        if sum(param) + len(param) - 1 == y: 
            ele = param.pop(0)
            new_grid = list(repeat(2, ele))
            while param:
                new_grid.append(1)
                ele = param.pop(0)
                new_grid.extend(list(repeat(2, ele)))

            for j, ele in enumerate(new_grid):
                _grid[j][i] = ele
            
    return _grid

In [204]:
temp_grid = first_round_topdown(grid, x, param_ver)

In [205]:
temp_grid = first_round_horizontal(temp_grid, y, param_hor)

In [157]:
temp_grid

[[0, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 0, 0, 0, 0, 0, 2, 1, 0, 2],
 [0, 0, 0, 0, 0, 0, 1, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2]]

### Second round

In [211]:
def param_to_regex(param):
    regex = r'[01]*'
    for ele in param:
        regex += r'2'*ele
        regex += r'[01]*'
    return re.compile(regex)

In [259]:
# top down
def second_round_topdown(grid, y, param_ver):
    _grid = deepcopy(grid)
    
    for i in range(y):
        line = _grid[i]
        param = param_ver[i]
        if 0 not in line: # line complete
            continue
        elif (y - line.count(1)) == sum(param): # line almost complete
            _grid[i] = [2 if l == 0 else l for l in line]
        else:
            line_str = ''.join(list(map(str, line)))
            pat = param_to_regex(param)
            if pat.search(line_str): # pattern already fulfilled 
                line_str = line_str.replace('0', '1')
                _grid[i] = list(map(int, list(line_str)))
            else: # pattern not fulfilled: search for possible solutions
                pass
        
    return _grid

In [260]:
third_grid = second_round_topdown(temp_grid, y, param_ver)

In [261]:
third_grid

[[1, 1, 1, 1, 1, 1, 2, 2, 1, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 0, 0, 0, 0, 0, 2, 1, 0, 2],
 [0, 0, 0, 0, 0, 0, 1, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [0, 0, 0, 0, 0, 0, 2, 2, 0, 2]]

In [264]:
# left to right
def second_round_left(grid, x, param_hor):
    _grid = deepcopy(grid)
    
    
    for i in range(x):
        line = [g[i] for g in _grid]
        print(line)
        param = param_hor[i]
        print(param)
        if 0 not in line: # line complete
            print('line complete\n')
            continue
        elif (y - line.count(1)) == sum(param): # line almost complete
            new_grid = [2 if l == 0 else l for l in line]
            for j, g in enumerate(_grid):
                g[i] = new_grid[j]
        else:
            line_str = ''.join(list(map(str, line)))
            pat = param_to_regex(param)
            if pat.search(line_str): # pattern already fulfilled 
                line_str = line_str.replace('0', '1')
                new_grid = list(map(int, list(line_str)))
                for j, g in enumerate(_grid):
                    g[i] = new_grid[j]
            else: # pattern not fulfilled: search for possible solutions
                pass
        
        #for j, g in enumerate(_grid):
         #   g[i] = new_grid[j]
        
        pprint(_grid)
        print('\n')
    return _grid

In [265]:
fourth_grid = second_round_left(third_grid, x, param_hor)

[1, 0, 0, 0, 2, 2, 0, 0, 2, 0]
[2, 1]
[[1, 1, 1, 1, 1, 1, 2, 2, 1, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 1, 0, 2],
 [1, 0, 0, 0, 0, 0, 1, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2]]


[1, 0, 0, 0, 2, 2, 0, 0, 2, 0]
[3, 1]
[[1, 1, 1, 1, 1, 1, 2, 2, 1, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 1, 0, 2],
 [1, 0, 0, 0, 0, 0, 1, 2, 0, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2]]


[1, 0, 0, 0, 2, 2, 0, 0, 2, 0]
[3, 1]
[[1, 1, 1, 1, 1, 1, 2, 2, 1, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 0, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 1

In [266]:
fourth_grid

[[1, 1, 1, 1, 1, 1, 2, 2, 1, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 2, 1],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 1, 2, 2],
 [1, 0, 0, 0, 0, 0, 1, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 0, 0, 0, 0, 0, 2, 2, 2, 2]]